# FlowSlide 调试诊断笔记本

这个笔记本用于诊断FlowSlide项目的导入和启动问题。

## 问题描述
- FlowSlide项目已完成从LandPPT的重新品牌化
- 端口已从8000更改为5000  
- 本地启动时遇到问题，需要诊断代码问题

## 调试计划
1. 环境设置和初始检查
2. 文件系统验证
3. Python路径配置
4. 模块导入测试
5. 错误处理和追踪分析
6. 应用对象检查

## 1. 环境设置和初始检查

首先检查Python环境和基础信息

In [1]:
import sys
import os
from pathlib import Path

print("=== FlowSlide 环境诊断 ===\n")

# Python版本和路径
print("1. Python环境:")
print(f"   版本: {sys.version}")
print(f"   可执行文件: {sys.executable}")
print(f"   平台: {sys.platform}")

# 当前工作目录
print(f"\n2. 工作目录:")
print(f"   当前: {os.getcwd()}")

# 设置项目根目录
project_root = Path.cwd()
print(f"   项目根: {project_root}")

# 切换到项目目录
if "landppt-integrated" not in str(project_root):
    project_root = project_root / "landppt-integrated"
    if project_root.exists():
        os.chdir(project_root)
        print(f"   ✅ 切换到: {project_root}")
    else:
        print(f"   ❌ 项目目录不存在: {project_root}")

print(f"   最终工作目录: {os.getcwd()}")

=== FlowSlide 环境诊断 ===

1. Python环境:
   版本: 3.11.3 (tags/v3.11.3:f3909b8, Apr  4 2023, 23:49:59) [MSC v.1934 64 bit (AMD64)]
   可执行文件: d:\Programs\Python\python.exe
   平台: win32

2. 工作目录:
   当前: e:\pyprojects\try1\landppt-integrated
   项目根: e:\pyprojects\try1\landppt-integrated
   最终工作目录: e:\pyprojects\try1\landppt-integrated


## 2. 文件系统验证

检查关键文件和目录是否存在

In [2]:
# 关键文件列表
critical_files = [
    "src/flowslide/__init__.py",
    "src/flowslide/main.py",
    "src/flowslide/core/simple_config.py",
    "src/flowslide/database/database.py",
    "src/flowslide/api/flowslide_api.py",
    "requirements.txt",
    "pyproject.toml"
]

# 关键目录列表
critical_dirs = [
    "src",
    "src/flowslide",
    "src/flowslide/core",
    "src/flowslide/database",
    "src/flowslide/api",
    "data"
]

print("3. 文件系统检查:")
print("\n   📁 关键目录:")
all_dirs_exist = True
for dir_path in critical_dirs:
    exists = os.path.exists(dir_path)
    print(f"      {dir_path}: {'✅' if exists else '❌'}")
    if not exists:
        all_dirs_exist = False

print("\n   📄 关键文件:")
all_files_exist = True
for file_path in critical_files:
    exists = os.path.exists(file_path)
    print(f"      {file_path}: {'✅' if exists else '❌'}")
    if not exists:
        all_files_exist = False

# 创建data目录如果不存在
if not os.path.exists("data"):
    os.makedirs("data")
    print("\n   ✅ 创建了data目录")

print(f"\n   📊 总结:")
print(f"      目录: {'全部存在' if all_dirs_exist else '有缺失'}")
print(f"      文件: {'全部存在' if all_files_exist else '有缺失'}")

3. 文件系统检查:

   📁 关键目录:
      src: ✅
      src/flowslide: ✅
      src/flowslide/core: ✅
      src/flowslide/database: ✅
      src/flowslide/api: ✅
      data: ✅

   📄 关键文件:
      src/flowslide/__init__.py: ✅
      src/flowslide/main.py: ✅
      src/flowslide/core/simple_config.py: ✅
      src/flowslide/database/database.py: ✅
      src/flowslide/api/flowslide_api.py: ✅
      requirements.txt: ✅
      pyproject.toml: ✅

   📊 总结:
      目录: 全部存在
      文件: 全部存在


## 3. Python路径配置

配置Python模块搜索路径和环境变量

In [3]:
# 配置Python路径
src_path = os.path.join(os.getcwd(), "src")
if src_path not in sys.path:
    sys.path.insert(0, src_path)
    print(f"✅ 添加src到Python路径: {src_path}")
else:
    print(f"ℹ️  src已在Python路径中: {src_path}")

# 设置环境变量
os.environ.setdefault("DATABASE_URL", "sqlite:///./data/flowslide.db")
os.environ.setdefault("PORT", "5000")
os.environ.setdefault("DEBUG", "True")

print(f"\n4. 环境变量配置:")
print(f"   DATABASE_URL: {os.environ.get('DATABASE_URL')}")
print(f"   PORT: {os.environ.get('PORT')}")
print(f"   DEBUG: {os.environ.get('DEBUG')}")

print(f"\n5. Python路径:")
print(f"   当前sys.path前3项:")
for i, path in enumerate(sys.path[:3]):
    print(f"      {i+1}. {path}")

✅ 添加src到Python路径: e:\pyprojects\try1\landppt-integrated\src

4. 环境变量配置:
   DATABASE_URL: sqlite:///./data/flowslide.db
   PORT: 5000
   DEBUG: True

5. Python路径:
   当前sys.path前3项:
      1. e:\pyprojects\try1\landppt-integrated\src
      2. e:\pyprojects\try1\landppt-integrated
      3. d:\Programs\Python\python311.zip


## 4. 模块导入测试

逐步测试FlowSlide模块导入

In [7]:
import traceback

print("6. 逐步导入测试:")

# 测试1: 基础flowslide包
try:
    print("\n   6.1 导入flowslide包...")
    import flowslide
    print("   ✅ flowslide包导入成功")
    print(f"      版本: {getattr(flowslide, '__version__', '未知')}")
except Exception as e:
    print(f"   ❌ flowslide包导入失败: {e}")
    traceback.print_exc()

# 测试2: 配置模块
try:
    print("\n   6.2 导入simple_config...")
    from flowslide.core import simple_config
    print("   ✅ simple_config导入成功")
    print(f"      数据库URL: {simple_config.DATABASE_URL}")
    print(f"      端口: {simple_config.app_config.port}")
except Exception as e:
    print(f"   ❌ simple_config导入失败: {e}")
    traceback.print_exc()

# 测试3: 数据库模块
try:
    print("\n   6.3 导入database...")
    from flowslide.database import database
    print("   ✅ database导入成功")
    print(f"      数据库类型: {getattr(database, 'DATABASE_TYPE', '未知')}")
except Exception as e:
    print(f"   ❌ database导入失败: {e}")
    traceback.print_exc()

# 测试4: API模块
try:
    print("\n   6.4 导入flowslide_api...")
    from flowslide.api import flowslide_api
    print("   ✅ flowslide_api导入成功")
except Exception as e:
    print(f"   ❌ flowslide_api导入失败: {e}")
    traceback.print_exc()

6. 逐步导入测试:

   6.1 导入flowslide包...
   ✅ flowslide包导入成功
      版本: 0.1.0

   6.2 导入simple_config...
   ✅ simple_config导入成功
      数据库URL: sqlite:///./data/flowslide.db
      端口: 5000

   6.3 导入database...
   ✅ database导入成功
      数据库类型: sqlite

   6.4 导入flowslide_api...


Tavily API key not found in configuration
Failed to initialize enhanced research service: 'AIConfig' object has no attribute 'get_provider_config'


   ✅ flowslide_api导入成功


## 5. 主应用导入测试

测试FlowSlide主应用模块

In [13]:
# 测试5: 主应用模块
try:
    print("   6.5 导入main模块...")
    from flowslide import main
    print("   ✅ main模块导入成功")
    
    print("   6.6 获取app对象...")
    app = main.app
    print(f"   ✅ app对象获取成功: {type(app)}")
    print(f"      应用标题: {app.title}")
    print(f"      应用版本: {app.version}")
    print(f"      文档URL: {app.docs_url}")
    
    # 检查路由
    print(f"      路由数量: {len(app.routes)}")
    print("      主要路由:")
    for route in app.routes[:5]:  # 显示前5个路由
        if hasattr(route, 'path'):
            print(f"         {route.path}")
    
except Exception as e:
    print(f"   ❌ main模块导入失败: {e}")
    print("\n详细错误追踪:")
    traceback.print_exc()

print("\n=== 导入测试完成 ===")

   6.5 导入main模块...


INFO:flowslide.main:Mounted temp directory: e:\pyprojects\try1\landppt-integrated\temp


   ✅ main模块导入成功
   6.6 获取app对象...
   ✅ app对象获取成功: <class 'fastapi.applications.FastAPI'>
      应用标题: FlowSlide API
      应用版本: 0.1.0
      文档URL: /docs
      路由数量: 162
      主要路由:
         /openapi.json
         /docs
         /docs/oauth2-redirect
         /redoc
         /auth/login

=== 导入测试完成 ===


## 6. 启动测试

如果导入成功，尝试启动服务器

In [ ]:
# 检查uvicorn是否可用
try:
    import uvicorn
    print(f"✅ uvicorn版本: {uvicorn.__version__}")
    
    # 如果前面的导入都成功了，app应该已经可用
    if 'app' in locals():
        print("\n🚀 app对象可用，可以启动服务器")
        print("启动命令:")
        print("   uvicorn.run(app, host='127.0.0.1', port=5000)")
        print("\n📍 访问地址:")
        print("   主页: http://127.0.0.1:5000")
        print("   文档: http://127.0.0.1:5000/docs")
        print("   Web: http://127.0.0.1:5000/web")
        
        # 注意：在notebook中不实际启动服务器，因为会阻塞
        print("\n💡 提示：在notebook中不会实际启动服务器")
        print("   请在终端中运行: python start_simple_fixed.py")
    else:
        print("❌ app对象不可用，无法启动")
        
except ImportError:
    print("❌ uvicorn未安装")
    print("💡 安装命令: pip install uvicorn")
except Exception as e:
    print(f"❌ uvicorn检查失败: {e}")

print("\n=== 诊断完成 ===")
print("如果所有导入都成功，问题可能在于:")
print("1. 依赖包缺失 (检查requirements.txt)")
print("2. 数据库初始化问题")
print("3. 端口占用")
print("4. 权限问题")

In [ ]:
import traceback

print("6. 逐步模块导入测试:")

# 测试导入列表
import_tests = [
    ("flowslide", "import flowslide"),
    ("flowslide.core.simple_config", "from flowslide.core import simple_config"),
    ("flowslide.database.database", "from flowslide.database import database"),
    ("flowslide.api.flowslide_api", "from flowslide.api import flowslide_api"),
    ("flowslide.main", "from flowslide import main"),
]

successful_imports = []
failed_imports = []

for name, import_stmt in import_tests:
    try:
        print(f"\n   🧪 测试: {name}")
        print(f"       命令: {import_stmt}")
        exec(import_stmt)
        print(f"       ✅ 成功")
        successful_imports.append(name)
    except Exception as e:
        print(f"       ❌ 失败: {e}")
        failed_imports.append((name, str(e)))

print(f"\n📊 导入测试总结:")
print(f"   ✅ 成功: {len(successful_imports)}/{len(import_tests)}")
print(f"   ❌ 失败: {len(failed_imports)}/{len(import_tests)}")

if successful_imports:
    print(f"\n   成功的导入:")
    for name in successful_imports:
        print(f"      • {name}")

if failed_imports:
    print(f"\n   失败的导入:")
    for name, error in failed_imports:
        print(f"      • {name}: {error}")

## 5. 错误详细分析

如果有导入失败，显示详细的错误追踪

In [ ]:
# 详细错误分析
if failed_imports:
    print("🔍 详细错误分析:")
    
    for name, error in failed_imports:
        print(f"\n{'='*50}")
        print(f"❌ 模块: {name}")
        print(f"错误: {error}")
        print(f"{'='*50}")
        
        # 重新尝试导入以获取完整追踪
        try:
            if name == "flowslide":
                import flowslide
            elif name == "flowslide.core.simple_config":
                from flowslide.core import simple_config
            elif name == "flowslide.database.database":
                from flowslide.database import database
            elif name == "flowslide.api.flowslide_api":
                from flowslide.api import flowslide_api
            elif name == "flowslide.main":
                from flowslide import main
        except Exception as e:
            print("完整错误追踪:")
            traceback.print_exc()
            print("")
else:
    print("🎉 所有导入都成功，没有错误需要分析！")

## 6. 应用对象检查

如果主模块导入成功，检查FastAPI应用对象

In [ ]:
# 应用对象检查
if "flowslide.main" in successful_imports:
    try:
        print("🔍 FastAPI应用检查:")
        
        # 获取app对象
        from flowslide.main import app
        print(f"   ✅ 应用对象类型: {type(app)}")
        print(f"   📋 应用标题: {app.title}")
        print(f"   📖 应用描述: {app.description[:50]}...")
        print(f"   🔢 应用版本: {app.version}")
        
        # 检查路由
        print(f"\n📍 注册的路由:")
        route_count = 0
        for route in app.routes:
            if hasattr(route, 'path') and hasattr(route, 'methods'):
                route_count += 1
                if route_count <= 5:  # 只显示前5个路由
                    methods = list(route.methods) if route.methods else ['GET']
                    print(f"      {methods[0]} {route.path}")
        
        if route_count > 5:
            print(f"      ... 和其他 {route_count - 5} 个路由")
        
        print(f"   📊 总路由数: {route_count}")
        
        # 检查依赖
        print(f"\n📦 依赖检查:")
        dependencies = ['uvicorn', 'fastapi', 'sqlalchemy', 'pydantic']
        for dep in dependencies:
            try:
                module = __import__(dep)
                version = getattr(module, '__version__', '未知')
                print(f"      ✅ {dep}: {version}")
            except ImportError:
                print(f"      ❌ {dep}: 未安装")
        
        print(f"\n🚀 应用准备状态: 可以尝试启动!")
        print(f"💡 启动命令: python start_simple_fixed.py")
        
    except Exception as e:
        print(f"❌ 应用对象检查失败: {e}")
        traceback.print_exc()
else:
    print("❌ flowslide.main 导入失败，无法检查应用对象")

print(f"\n{'='*60}")
print(f"🏁 FlowSlide 诊断完成")
print(f"{'='*60}")